## Import Libraries

In [74]:
import os
from crewai import Agent, Task, Crew
from crewai.flow.flow import Flow, start, listen  
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import DuckDuckGoSearchRun



## Load the  LLM


In [75]:
from langchain_community.chat_models.llamacpp import ChatLlamaCpp

llm = ChatLlamaCpp(
    model_path="llama-3-8b-function-calling.Q4_K_M.gguf",
    n_ctx=1024,
    n_threads=4,
    temperature=0.0,
    verbose=True
)




llama_model_load_from_file_impl: using device Metal (Apple M4) - 10916 MiB free
gguf_init_from_file_impl: invalid magic characters: 'Entr', expected 'GGUF'
llama_model_load: error loading model: llama_model_loader: failed to load model from llama-3-8b-function-calling.Q4_K_M.gguf
llama_model_load_from_file_impl: failed to load model


ValidationError: 1 validation error for ChatLlamaCpp
  Value error, Could not load Llama model from path: llama-3-8b-function-calling.Q4_K_M.gguf. Received error Failed to load model from file: llama-3-8b-function-calling.Q4_K_M.gguf [type=value_error, input_value={'model_path': 'llama-3-8...': 0.0, 'verbose': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

## Load Tools

In [ ]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool("DuckDuckGo Search")
def search_tool1(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    return DuckDuckGoSearchRun().run(query)



## Agent Definitions



In [ ]:
loader_agent = Agent(
    role="Loader Agent",
    goal="Search and retrieve electric consumption data for Indian states (2018-2023).",
    backstory="A research assistant who finds credible data sources from the web.",
    tools=[search_tool1],
    llm=llm,
    verbose=True
)

In [ ]:
from langchain_experimental.tools import PythonREPLTool


@tool("analyzer")
def search_tool2(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    return PythonREPLTool().run(query)


In [ ]:
analyzer_agent = Agent(
    role="Analyzer Agent",
    goal="Analyze the CSV to find top 3 states with highest average consumption.",
    backstory="A data analyst who uses pandas to extract patterns.",
    tools=[search_tool1],
    llm=llm,
    verbose=True
)
#This one loads CSV and analyzes with pandas.

In [ ]:
visualizer_agent = Agent(
    role="Visualizer Agent",
    goal="Create bar charts to show electricity consumption per year by state.",
    backstory="A matplotlib expert for Indian state electricity data.",
    tools=[search_tool2],
    llm=llm,
    verbose=True
)
#Makes visual plots of electricity data.

In [ ]:
summarizer_agent = Agent(
    role="Summarizer Agent",
    goal="Write a 2-3 sentence summary about which states consumed the most electricity and why.",
    backstory="A professional executive summary writer.",
    tools=[],
    llm=llm,
    verbose=True
)
#Pure LLM-based summary writer.



## Task Definitions

In [ ]:
load_task = Task(
    description="Search for electricity consumption datasets for Indian states (2018–2023).",
    agent=loader_agent,
    expected_output="A list of 3 URLs to government or trusted sources",
    input_type="input"
    )


analyze_task = Task(
    description="Load a CSV file, calculate average consumption, and print top 3 states.",
    agent=analyzer_agent,
    expected_output="A list of the top 3 Indian states with the highest average electricity consumption."
)

visualize_task = Task(
    description="Use matplotlib/seaborn to plot electricity consumption for each state.",
    agent=visualizer_agent,
    expected_output="A bar chart or line graph showing electricity consumption for each Indian state."
)

summarize_task = Task(
    description="Write a summary of insights from the data and visualizations in 2-3 sentences.",
    agent=summarizer_agent,
    expected_output="A concise textual summary of the most important insights from the analysis and visualizations."
)


## Flow Definition

In [ ]:
from crewai import Crew, Process

#  Step 1: Run the loader agent first (sequential)
loader_crew = Crew(
    agents=[loader_agent],
    tasks=[load_task],
    process=Process.sequential
)

#  Get the context (output from loader_agent)
context = loader_crew.kickoff(inputs={"input": "Search for electricity consumption datasets for Indian states (2018–2023). Focus on government or trusted sources. Return top 3 links."})



#  Step 2: Set the next tasks to run in parallel
analyze_task.async_execution = True
visualize_task.async_execution = True
summarize_task.async_execution = True

#  Step 3: Run the 3 agents in parallel using shared context
main_crew = Crew(
    agents=[analyzer_agent, visualizer_agent, summarizer_agent],
    tasks=[analyze_task, visualize_task, summarize_task],
    process=Process.parallel
)

#  Execute the second crew and collect results
final_result = main_crew.kickoff(context=context)

# Print the final combined output
print(final_result)



╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Loader Agent                                                                                            │
│                                                                                                                 │
│  Task: Search for electricity consumption datasets for Indian states (2018–2023).                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=verbose=False client=<llama_cpp.llama.Llama object at 0x156cea630> model_path='/Users/pranjalkamboj/Desktop/projects/multi-agents-and-tools-using-Crewai/openchat-3.5-0106.Q4_K_M.gguf' n_ctx=1024 n_threads=4 model_kwargs={}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

##  Run and Show Result

In [ ]:
crew = Crew(flow=flow, verbose=True)
result = crew.kickoff()

print("\n Final Result:ssssss
